<a href="https://colab.research.google.com/github/yxmauw/GA_Capstone/blob/main/Capstone.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
# libraries to read github files
import zipfile
import os

In [ ]:
# shell command, downloads to current directory
# ignore SSL certificate error if website has expired SSL certificate
!wget --no-check-certificate \ 
    "https://github.com/yxmauw/GA_Capstone/archive/refs/heads/main.zip" \
    -O "/glaucoma.zip"


zip_ref = zipfile.ZipFile('/glaucoma.zip', 'r') #Opens the zip file in read mode
zip_ref.extractall('/tmp') #Extracts the files into the /tmp folder
zip_ref.close()